In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

In [61]:
#import all my files to dataframes
movie_gross_df = pd.read_csv('data/bom.movie_gross.csv')
rt_info_df = pd.read_csv('data/rt.movie_info.tsv', sep='\t')
rt_reviews_df = pd.read_csv('data/rt.reviews.tsv', sep='\t')
tmdb_movies_df = pd.read_csv('data/tmdb.movies.csv')
budgets_df = pd.read_csv('data/tn.movie_budgets.csv')

# Movie Gross

In [19]:
movie_gross_df.head()

,title,studio,domestic_gross,foreign_gross,year
0,Toy Story 3,BV,415000000.0,652000000,2010
1,Alice in Wonderland (2010),BV,334200000.0,691300000,2010
2,Harry Potter and the Deathly Hallows Part 1,WB,296000000.0,664300000,2010
3,Inception,WB,292600000.0,535700000,2010
4,Shrek Forever After,P/DW,238700000.0,513900000,2010


In [18]:
movie_gross_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3387 entries, 0 to 3386
Data columns (total 5 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   title           3387 non-null   object 
 1   studio          3382 non-null   object 
 2   domestic_gross  3359 non-null   float64
 3   foreign_gross   2037 non-null   object 
 4   year            3387 non-null   int64  
dtypes: float64(1), int64(1), object(3)
memory usage: 132.4+ KB


In [21]:
movie_gross_df['year'].unique()

array([2010, 2011, 2012, 2013, 2014, 2015, 2016, 2017, 2018], dtype=int64)

Initial observations: This dataset includes both foreign and domestic gross for movies from 2010-2018. Will be useful for looking at what types of movies make the most money, and looking at trends over time. There are a few missing values, but this data should be easy to clean.

In [32]:
#Fill null values in the studio column with 'Other'
movie_gross_df['studio'].fillna(value='Other', inplace=True)

In [37]:
#Inspect data where domestic_gross is null
movie_gross_df.loc[movie_gross_df['domestic_gross'].isna() == True]

,title,studio,domestic_gross,foreign_gross,year
230,It's a Wonderful Afterlife,UTV,NaN,1300000,2010
298,Celine: Through the Eyes of the World,Sony,NaN,119000,2010
302,White Lion,Scre.,NaN,99600,2010
306,Badmaash Company,Yash,NaN,64400,2010
327,Aashayein (Wishes),Relbig.,NaN,3800,2010
537,Force,FoxS,NaN,4800000,2011
713,Empire of Silver,NeoC,NaN,19000,2011
871,Solomon Kane,RTWC,NaN,19600000,2012
928,The Tall Man,Imag.,NaN,5200000,2012
933,Keith Lemon: The Film,Other,NaN,4000000,2012


At a glance, I am reasonably sure that none of these movies are going to be glaring omissions, and since this is less than 1% of the data, I feel comfortable just dropping these rows entirely.

In [42]:
#Drop offending rows
movie_gross_df.dropna(subset=['domestic_gross'], inplace=True)

In [43]:
#Inspect rows where foreign_gross is null
movie_gross_df.loc[movie_gross_df['foreign_gross'].isna() == True]

,title,studio,domestic_gross,foreign_gross,year
222,Flipped,WB,1800000.0,NaN,2010
254,The Polar Express (IMAX re-issue 2010),WB,673000.0,NaN,2010
267,Tiny Furniture,IFC,392000.0,NaN,2010
269,Grease (Sing-a-Long re-issue),Par.,366000.0,NaN,2010
280,Last Train Home,Zeit.,288000.0,NaN,2010
...,...,...,...,...,...
3382,The Quake,Magn.,6200.0,NaN,2018
3383,Edward II (2018 re-release),FM,4800.0,NaN,2018
3384,El Pacto,Sony,2500.0,NaN,2018
3385,The Swan,Synergetic,2400.0,NaN,2018


I am really not sure what to do with this data -- it represents about 1/3 of the dataset, so dropping that much data really isn't an option. My gut instinct says that these films did not get a foreign release at all, which is why the data is missing. Additionally, I'm not sure if I will want to even use the foreign_gross data. So for now, I am going to leave the data as-is, and reconsider this question later if it becomes relevent to our analysis.

In [44]:
movie_gross_df.info()
#Other than the foreign_gross column, we have dealt with all null values.

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3359 entries, 0 to 3386
Data columns (total 5 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   title           3359 non-null   object 
 1   studio          3359 non-null   object 
 2   domestic_gross  3359 non-null   float64
 3   foreign_gross   2009 non-null   object 
 4   year            3359 non-null   int64  
dtypes: float64(1), int64(1), object(3)
memory usage: 157.5+ KB


In [45]:
#Save cleaned data to a new csv file
movie_gross_df.to_csv('Movie Gross Info.csv', index=False)

# Rotten Tomatoes Info

In [104]:
rt_info_df.head()

,id,synopsis,rating,genre,director,writer,theater_date,dvd_date,currency,box_office,runtime,studio
0,1,"This gritty, fast-paced, and innovative police...",R,Action and Adventure|Classics|Drama,William Friedkin,Ernest Tidyman,1971-10-09,"Sep 25, 2001",NaN,NaN,104 minutes,NaN
1,3,"New York City, not-too-distant-future: Eric Pa...",R,Drama|Science Fiction and Fantasy,David Cronenberg,David Cronenberg|Don DeLillo,2012-08-17,"Jan 1, 2013",$,"600,000",108 minutes,Entertainment One
2,5,Illeana Douglas delivers a superb performance ...,R,Drama|Musical and Performing Arts,Allison Anders,Allison Anders,1996-09-13,"Apr 18, 2000",NaN,NaN,116 minutes,NaN
3,6,Michael Douglas runs afoul of a treacherous su...,R,Drama|Mystery and Suspense,Barry Levinson,Paul Attanasio|Michael Crichton,1994-12-09,"Aug 27, 1997",NaN,NaN,128 minutes,NaN
4,7,NaN,NR,Drama|Romance,Rodney Bennett,Giles Cooper,NaT,NaN,NaN,NaN,200 minutes,NaN


In [100]:
rt_info_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1560 entries, 0 to 1559
Data columns (total 12 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   id            1560 non-null   int64         
 1   synopsis      1498 non-null   object        
 2   rating        1557 non-null   object        
 3   genre         1552 non-null   object        
 4   director      1361 non-null   object        
 5   writer        1111 non-null   object        
 6   theater_date  1201 non-null   datetime64[ns]
 7   dvd_date      1201 non-null   object        
 8   currency      340 non-null    object        
 9   box_office    340 non-null    object        
 10  runtime       1530 non-null   object        
 11  studio        494 non-null    object        
dtypes: datetime64[ns](1), int64(1), object(10)
memory usage: 146.4+ KB


Well, every row has at least one null value, so that's fun. I can see immediately that this extends way beyond the 2010-2018 range of the previous dataset. Since my business question is about making recommendations for the future, I think in general I won't be interested in movies prior to 2010 anyway, so I will begin by splitting this data into pre- and post-2010 dataframes. I want to retain the pre-2010 data for now, because I think it might possibly be interesting to recommend remakes of some popular older films, but that is based on my personal feeling that established properties tend to do better at the box office, so we will see if the data actually supports that before I do any more with that data.

In [76]:
#Convert theater_date to datetime format
rt_info_df['theater_date'] = pd.to_datetime(rt_info_df['theater_date'])

In [86]:
#Create new dataframe with only post-2010 movies
rt_recent_info_df = rt_info_df.loc[rt_info_df['theater_date'] >= '2010-01-01']

In [87]:
# Create new dataframe with pre-2010 movies
rt_old_info_df = rt_info_df.loc[rt_info_df['theater_date'] < '2010-01-01']

In [88]:
#Save pre-2010 data as a CSV in case I want it later
rt_old_info_df.to_csv('Rotten Tomatoes Info Pre 2010.csv', index=False)

In [92]:
#Save post-2010 data as a CSV
rt_recent_info_df.to_csv('Rotten Tomatoes Info Post 2010.csv', index=False)

Good news is that there is much less missing data in this dataset. However, it is extremely small (could be a good thing or a bad thing--hard to tell at this point, but it is definitely much smaller than my gross data). I also just realized that this dataset is missing the actual movie titles--my guess is those come from the other Rotten Tomatoes dataset.
For now, I am going to ignore the null values in this dataset. I'm not sure exactly what information will be the most valuable, so I don't want to remove or change this data until I am sure I need it.

# Rotten Tomato Reviews

In [93]:
rt_reviews_df.head()

,id,review,rating,fresh,critic,top_critic,publisher,date
0,3,A distinctly gallows take on contemporary fina...,3/5,fresh,PJ Nabarro,0,Patrick Nabarro,"November 10, 2018"
1,3,It's an allegory in search of a meaning that n...,NaN,rotten,Annalee Newitz,0,io9.com,"May 23, 2018"
2,3,... life lived in a bubble in financial dealin...,NaN,fresh,Sean Axmaker,0,Stream on Demand,"January 4, 2018"
3,3,Continuing along a line introduced in last yea...,NaN,fresh,Daniel Kasman,0,MUBI,"November 16, 2017"
4,3,... a perverse twist on neorealism...,NaN,fresh,NaN,0,Cinema Scope,"October 12, 2017"


In [94]:
rt_reviews_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 54432 entries, 0 to 54431
Data columns (total 8 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   id          54432 non-null  int64 
 1   review      48869 non-null  object
 2   rating      40915 non-null  object
 3   fresh       54432 non-null  object
 4   critic      51710 non-null  object
 5   top_critic  54432 non-null  int64 
 6   publisher   54123 non-null  object
 7   date        54432 non-null  object
dtypes: int64(2), object(6)
memory usage: 3.3+ MB


Interestingly, this Rotten Tomatoes Dataset also doesn't have movie titles......
That may make the Rotten Tomatoes data difficult to use. I'm not going to make any changes to this data for now, and we will see how my inquiries shake out.

# TMDB Data

In [105]:
tmdb_movies_df.head()

,Unnamed: 0,genre_ids,id,original_language,original_title,popularity,release_date,title,vote_average,vote_count
0,0,"[12, 14, 10751]",12444,en,Harry Potter and the Deathly Hallows: Part 1,33.533,2010-11-19,Harry Potter and the Deathly Hallows: Part 1,7.7,10788
1,1,"[14, 12, 16, 10751]",10191,en,How to Train Your Dragon,28.734,2010-03-26,How to Train Your Dragon,7.7,7610
2,2,"[12, 28, 878]",10138,en,Iron Man 2,28.515,2010-05-07,Iron Man 2,6.8,12368
3,3,"[16, 35, 10751]",862,en,Toy Story,28.005,1995-11-22,Toy Story,7.9,10174
4,4,"[28, 878, 12]",27205,en,Inception,27.920,2010-07-16,Inception,8.3,22186


In [106]:
tmdb_movies_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 26517 entries, 0 to 26516
Data columns (total 10 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Unnamed: 0         26517 non-null  int64  
 1   genre_ids          26517 non-null  object 
 2   id                 26517 non-null  int64  
 3   original_language  26517 non-null  object 
 4   original_title     26517 non-null  object 
 5   popularity         26517 non-null  float64
 6   release_date       26517 non-null  object 
 7   title              26517 non-null  object 
 8   vote_average       26517 non-null  float64
 9   vote_count         26517 non-null  int64  
dtypes: float64(2), int64(3), object(5)
memory usage: 2.0+ MB


This data appears not to have any null values--it's possible/likely that there are placeholder type values instead--I would be shocked if the data was this clean right out of the gate. 
The genres are only id values--I will need to pull the actual genres out of TMDB if I want to use them. Considering TMDB apparently has an excellent API, I might be inclined to just grab any data I might want from there instead.

# Budgets 

In [107]:
budgets_df.head()

,id,release_date,movie,production_budget,domestic_gross,worldwide_gross
0,1,"Dec 18, 2009",Avatar,"$425,000,000","$760,507,625","$2,776,345,279"
1,2,"May 20, 2011",Pirates of the Caribbean: On Stranger Tides,"$410,600,000","$241,063,875","$1,045,663,875"
2,3,"Jun 7, 2019",Dark Phoenix,"$350,000,000","$42,762,350","$149,762,350"
3,4,"May 1, 2015",Avengers: Age of Ultron,"$330,600,000","$459,005,868","$1,403,013,963"
4,5,"Dec 15, 2017",Star Wars Ep. VIII: The Last Jedi,"$317,000,000","$620,181,382","$1,316,721,747"


In [108]:
budgets_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5782 entries, 0 to 5781
Data columns (total 6 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   id                 5782 non-null   int64 
 1   release_date       5782 non-null   object
 2   movie              5782 non-null   object
 3   production_budget  5782 non-null   object
 4   domestic_gross     5782 non-null   object
 5   worldwide_gross    5782 non-null   object
dtypes: int64(1), object(5)
memory usage: 271.2+ KB


In [110]:
budgets_df['release_date'].min()

'Apr 1, 1975'

Loving this dataset -- no missing values, super straightforward and clean. I think the only data I am really wishing I had here is genres.

# Possible Questions from Looking at This Data

1. Compare budget to gross to see what types of movies tend to provide the best bang for your buck (great for a startup studio). Also look at movies that lost money and/or broke even to determine what types of movies should be avoided.
2. Analyze the top grossing movies--what do they have in common?
3. Is there a correlation between positive reviews and gross? Is there a correlation between positive reviews and ROI?